# 自动补全联系人

In [108]:
import redis
import time
import datetime
import bisect
import uuid
import math
conn = redis.Redis(decode_responses=True)

In [7]:
def add_update_contact(conn, user, contact):
    ac_list = 'recent:'+user
    pipe = conn.pipeline(True)
    pipe.lrem(ac_list, 0, contact)
    pipe.lpush(ac_list, contact)
    pipe.ltrim(ac_list, 0, 10)
    pipe.execute()

In [16]:
def fetch_autocomplete_list(conn, user, prefix):
    candidates = list(conn.lrange('recent:'+user, 0, -1))
    matches = []
    for candidate in candidates:
        if candidate.lower().startswith(prefix):
            matches.append(candidate)
    return matches

In [17]:
add_update_contact(conn,'user1','cont')

In [18]:
fetch_autocomplete_list(conn,'user1','con')

['cont']

In [20]:
# 通过插入在有序列表中插入前缀之前的一个字符串和之后的一个字符串来获取中间满足条件的字符串
validate_characters = '`abcdefghijklmnopqrstuvwxyz{'

def find_prefix_range(prefix):
    posn = bisect.bisect_left(validate_characters, prefix[-1:])
    suffix = validate_characters[(posn or 1) - 1]
    return prefix[:-1]+suffix+'{', prefix+'}'


In [30]:
def autocomplete_on_prefix(conn, guild, prefix):
    start, end = find_prefix_range(prefix)
    identifier = str(uuid.uuid4())
    start += identifier
    end += identifier
    zset_name = 'members:'+guild
    
    conn.zadd(zset_name, {start: 0, end: 0})
    pipe = conn.pipeline(True)
    while True:
        try:
            pipe.watch(zset_name)
            sindex = pipe.zrank(zset_name, start)
            eindex = pipe.zrank(zset_name, end)
            erange = min(sindex + 9, eindex-2)
            pipe.multi()
            
            pipe.zrem(zset_name, start, end)
            pipe.zrange(zset_name, sindex, erange)
            items = pipe.execute()[-1]
            break
        except redis.exceptions.WatchError:
            continue
    return [item for item in items if '{' not in item]

In [23]:
def join_guild(conn, guild, user):
    conn.zadd('members:'+guild, {user:0})
    
def leave_guild(conn, guild, user):
    conn.zrem('members:'+guild, user)

In [26]:
join_guild(conn, 'guild1', 'use2')

conn.zrange('members:guild1',0,-1, withscores=True)

In [32]:
autocomplete_on_prefix(conn, 'guild1', 'user')

['user1', 'user2']

# 锁

In [107]:
def acquire_lock(conn, lockname, acquire_timeout=10):
    identifier = str(uuid.uuid4())
    
    end = time.time() + acquire_timeout
    while time.time() < end:
        if conn.setnx('lock:'+lockname, identifier):
            return identifier
        time.sleep(.001)
    return False

def release_lock(conn, lockname, identifier):
    pipe = conn.pipeline(True)
    lockname = 'lock:' + lockname
    
    while True:
        try:
            pipe.watch(lockname)
            if pipe.get(lockname) == identifier:
                pipe.multi()
                pipe.delete(lockname)
                pipe.execute()
                return True
            pipe.unwatch()
            break
        except redis.exceptions.RedisError:
            pass
        return False

In [122]:
def purchase_item_with_lock(conn, buyerid, itemid, sellerid):
    buyer = 'users:%s' % buyerid
    seller = 'users:%s' % sellerid
    item = '%s.%s' %(itemid, sellerid)
    inventory = 'inventory:%s' %buyerid
    
    #locked = acquire_lock(conn, 'market:')
    locked = acquire_lock_with_timeout(conn, 'market:')
    if not locked:
        print('获取锁失败')
        return False
    pipe = conn.pipeline(True) 
    try:
        pipe.zscore('market:', item)
        pipe.hget(buyer, 'funds')
        price, funds = map(int,pipe.execute())
        if price is None or price > funds:
            return None
        pipe.hincrby(seller, 'funds' , int(price))
        pipe.hincrby(buyer, 'funds', int(-price))
        pipe.sadd(inventory, itemid)
        pipe.zrem('market:', item)
        pipe.execute()
        return True
    finally:
        release_lock(conn, 'market:', locked)

In [124]:
conn.zrange('market:', 0, -1, withscores=True)

[]

In [125]:
conn.smembers('inventory:2')

{'itemD', 'itemE', 'itemF', 'itemG', 'itemH', 'itemL'}

In [126]:
conn.smembers('inventory:1')

{'itemG', 'itemH'}

In [127]:
conn.hgetall('users:1')

{'name': 'jim', 'funds': '140'}

In [128]:
conn.hgetall('users:2')

{'name': 'tom', 'funds': '3'}

In [123]:
purchase_item_with_lock(conn, 2, 'itemL',1)

True

In [121]:
def acquire_lock_with_timeout(conn, lockname, acquire_timeout=10, lock_timeout=10):
    identifier = str(uuid.uuid4())
    lockname = 'lock:'+lockname
    lock_timeout = int(math.ceil(lock_timeout))
    
    end = time.time() + acquire_timeout
    while time.time() < end:
        if conn.setnx(lockname,  identifier):
            conn.expire(lockname, lock_timeout)
            return identifier
        elif not conn.ttl(lockname):
            conn.expire(lockname, lock_timeout)
        time.sleep(.001)
    return False
    

In [120]:
conn.get('lock:market:')